In [1]:
import numpy as np
import pandas as pd
import pickle
import glob
import os

# FastText 

In [ ]:
DATAPATH = './data/data_baseline_fasttext_starspace/'
import glob
files = glob.glob(DATAPATH+'*')
labels_dict = {}
for file in files:
    with open(file, 'r') as f:
        lines = f.readlines()
    f.close()
    labels = [line[-5:-1] for line in lines]
    labels_dict[file[40:-4]] = labels
#     f_w = open(file, 'w')
#     for line in lines:
#         if 'LIVE' in line[-9:]:
#             change = '__label__LIVE\n'
#         else:
#             change = '__label__DEAD\n'

#         new_line  = ''.join([line[:-9], change])
#         f_w.write(new_line)
#     f_w.close()

        

In [ ]:
with open('./data/data_baseline_fasttext_starspace/labels_splitted_data.pickle', 'wb') as f:
    pickle.dump(labels_dict, f)

In [104]:
import pickle
with open('./data/data_baseline_fasttext_starspace/labels_splitted_data.pickle', 'rb') as f:
    labels_dict = pickle.load(f)

In [ ]:
from sklearn.metrics import roc_auc_score
def auc_calculation(data_name, prob_file_name, labels_dict):
    true = (pd.Series(labels_dict[data_name]) == 'DEAD') *1
    
    with open(prob_file_name, 'r') as f:
        lines = f.readlines()
    prob_dead = []
    for line in lines:
        label_1, prob_1,label_2, prob_2 = line[:-2].split(' ')
        if 'DEAD' in label_1:
            prob_dead.append(float(prob_1))
        else:
            prob_dead.append(float(prob_2))
    return roc_auc_score(true, prob_dead)

# CNN Structure 

In [2]:
DATAPATH = './data/data_baseline_fasttext_starspace/'

import glob
files = glob.glob(DATAPATH+'*')

model_time = '12h'

     

In [106]:
files_model = [file for file in files if model_time in file]

In [107]:
files_model
train_data = []

def read_data(file_name, path = DATAPATH, label = labels_dict):
    data = []
    file = path+file_name
    with open(file, 'r') as f:
        lines = f.readlines()
    f.close()
    label_model = np.array(labels_dict[file_name[:-4]]) == 'DEAD'
    label_model = label_model*1
    i = 0
    for line in lines:
        data.append({'text':line[:-15], 'label':label_model[i]})
        i+=1
    return data

def data_loading(model_time):
    train_data = read_data('train_'+model_time+'.txt')
    test_data = read_data('test_'+model_time+'.txt')
    validation_data = read_data('val_'+model_time+'.txt')
    return train_data, test_data, validation_data

In [108]:
train_data, test_data, validation_data =  data_loading(model_time)

In [3]:
DATAPATH = './data/data_cnn/'
#data = np.load(DATAPATH+'all_data.npy')

vocabulary = np.load(os.path.join(DATAPATH, 'voc_100.npy'))
index = range(len(vocabulary))
voca_dict = dict(zip(vocabulary, index))

test_6h = np.load(os.path.join(DATAPATH,'test_6h.npy')).item()
train_6h = np.load(os.path.join(DATAPATH,'train_6h.npy')).item()
val_6h = np.load(os.path.join(DATAPATH,'val_6h.npy')).item()

In [4]:
def notes_combine(infor_all):
    data = infor_all['DATA'] 
    notes_per_sub = []
    max_len = []
    for sub in data:
        notes = []
        for note in sub:
            d = [item for sublist in note for item in sublist]
            notes.append(d )
            max_len.append(len(d))
        notes_per_sub.append(notes)
    del notes
    del d
    return notes_per_sub, max_len

def notes_padding(notes_all, note_len):
    notes_per_sub = []
    for sub in notes_all:
        notes = []
        for note in sub:
            num_padding = note_len - len(note)
            new_sentence = note + [0] * num_padding
            notes.append(new_sentence )
        notes_per_sub.append(notes)
    del notes
    return notes_per_sub
def data_dict_generate(label, data):
    result = []
    for (d, l) in zip(data, label):
        #print(l)
        result.append({'text':d, 'label':torch.LongTensor([int(l)])})
    return result


In [5]:
test_notes_6h, len_test = notes_combine(test_6h)
train_notes_6h , len_train= notes_combine(train_6h)
val_notes_6h, len_val = notes_combine(val_6h)
len_all = np.array(len_test + len_train + len_val)

number_of_notes_train = [len(x) for x in train_notes_6h]
index = np.argsort(np.array(number_of_notes_train))
train_notes_6h= np.array(train_notes_6h)[list(index)]

In [6]:
test_notes_6h_padded = notes_padding(test_notes_6h, len_all.max())
train_notes_6h_padded = notes_padding(train_notes_6h, len_all.max())
val_notes_6h_padded = notes_padding(val_notes_6h, len_all.max())

In [11]:
val_data = data_dict_generate(val_6h['MORTALITY_LABEL'], val_notes_6h_padded)
train_data = data_dict_generate(np.array(train_6h['MORTALITY_LABEL'])[index], train_notes_6h_padded)
test_data = data_dict_generate(test_6h['MORTALITY_LABEL'], test_notes_6h_padded)

In [115]:
# import matplotlib.pyplot as plt
# % %matplotlib inline
# plt.hist(len_all, bins= 30)

In [8]:
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        yield [source[index] for index in batch_indices]

# This is the iterator we use when we're evaluating our model. 
# It gives a list of batches that you can then iterate through.
def eval_iter(source, batch_size):
    batches = []
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while start < dataset_size - batch_size:
        start += batch_size
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        if len(batch) == batch_size:
            batches.append(batch)
        else:
            continue
        
    return batches

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    labels = []
    for dict in batch:
        vectors.append(dict["text"])
        labels.append(dict["label"])
    return vectors, labels

def pad_minibatch(vectors):
    length = max([len(x) for x in vectors])
    padded_v = []
    note_len = len(vectors[0][0])
    for before_padded in vectors:
        #print(before_padded)
        v = before_padded.copy()
        #print(type(v))
        for i in range(length - len(before_padded)):
            v.append([0]*note_len)
        padded_v.append(v)
    return Variable(torch.from_numpy(np.array(padded_v)).permute(1,2,0))

In [9]:
# This function outputs the accuracy on the dataset, we will use it during training.
import sklearn.metrics as metrics
def evaluate(model, loss_, data_iter):
    model.eval()
    correct = 0
    total = 0
    labels_all = []
    output_all = []
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        vectors = pad_minibatch(vectors)
        labels = Variable(torch.stack(labels).squeeze())
        
        hidden = model.init_hidden()
        output = model(vectors, hidden)
        loss = loss_(output, labels)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        predicted = predicted.numpy()
        labels = labels.data.numpy()
        correct += (predicted == labels).sum()
        labels_all += list(labels)
        output_all += list(output.data.numpy())
    auc = metrics.roc_auc_score(labels_all, output_all)
    return loss, correct / float(total), auc

# This function gives us the confusion matrix for all labels and the overall accuracy.
def evaluate_confusion(model, data_iter, lstm):
    model.eval()
    correct_all = 0
    correct = {}
    for lab in easy_label_map:
        correct[lab] = [0,0,0,0,0] #eses, esar, ptpt, ptbr, total
    total = 0
    for i in range(len(data_iter)):
        vectors, labels = get_batch(data_iter[i])
        vectors = Variable(torch.stack(vectors).squeeze())
        labels = Variable(torch.stack(labels).squeeze())
        
        if lstm:
            hidden, c_t = model.init_hidden()
            output, hidden = model(vectors, hidden, c_t)
        else:
            hidden = model.init_hidden()
            output, hidden = model(vectors, hidden)
        
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct_all += (predicted == labels).sum()
        
        for lab in easy_label_map:
            inds = (labels[:] == easy_label_map[lab]).nonzero().squeeze()
            for i in range(len(easy_label_map)):
                tmp =  torch.ones(len(inds.size())).long()*i
                correct[lab][i] += (predicted[inds] == tmp).sum()
            correct[lab][-1] += inds.size(0)
        
        confusion = {}
        for val in correct:
            confusion[val] = {v:correct[val][i] for i, v in enumerate(easy_label_map)}
        
    return confusion, correct_all / float(total)

In [10]:
import torch.optim as optim
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import random
import logging


class AttentionRNN(nn.Module):
    def __init__(self, config):
        super(AttentionRNN, self).__init__()
        self.batch_size = config['batch_size']
        n_classes = config['target_class']
        output_channel = config['output_channel']
        self.note_gru_hidden = config['note_gru_hidden']
        self.bidirection_gru = config['bidirection_gru']
        self.note_embedding = Convolutional_Embedding(config)
        self.note_gru = nn.GRU(output_channel, self.note_gru_hidden, bidirectional= self.bidirection_gru)
        if self.bidirection_gru:
            self.lin_out = nn.Linear(self.note_gru_hidden * 2, n_classes)
        else:
            self.lin_out = nn.Linear(self.note_gru_hidden, n_classes)
        #self.softmax_note = nn.Softmax()
        
    def forward(self, mini_batch, hidden_state):
        num_of_notes, num_of_words, batch_size = mini_batch.size()
        s = None
        for i in range(num_of_notes):
            _s = self.note_embedding(mini_batch[i,:,:].transpose(0,1))
            if (s is None):
                s = _s.unsqueeze(0)
            else:
                s = torch.cat((s,_s.unsqueeze(0)),0)
        #print('CNN: ', s.size())
                
        #packed_s = torch.nn.utils.rnn.pack_padded_sequence(s, length, batch_first=True)
        
        # (seq_len, batch, input_size),  (num_layers * num_directions, batch, hidden_size)
        out_note, _ =  self.note_gru(s, hidden_state) 
        #print('RNN: ',out_note.size())
        x = out_note[-1,:,:].squeeze()
        x = self.lin_out(x)
        return F.softmax(x)
    
    def init_hidden(self):
        if self.bidirection_gru == True:
            return Variable(torch.zeros(2, self.batch_size, self.note_gru_hidden))
        else:
            return Variable(torch.zeros(1, self.batch_size, self.note_gru_hidden))

        
class Convolutional_Embedding(nn.Module):
    def __init__(self, config):
        super(Convolutional_Embedding, self).__init__()
        words_dim = config['words_dim']
        self.embed_mode = config['embed_mode']
        
        output_channel = config['output_channel'] #
        filter_width = config['filter_width']
        
        
        vocab_size = config['vocab_size']
        self.word_embed = nn.Embedding(vocab_size, words_dim)
        #self.static_word_embed = nn.Embedding(vocab_size, words_dim)
        #self.nonstatic_word_embed = nn.Embedding(vocab_size, words_dim)

        #self.static_word_embed.weight.requires_grad = False
 
        input_channel = 1
    
        self.conv= nn.Conv2d(input_channel, output_channel, (filter_width, words_dim), padding=(filter_width - 1, 0))
        self.dropout = nn.Dropout(config['dropout'])
        
        #n_hidden = output_channel 

        #self.combined_feature_vector = nn.Linear(n_hidden, n_hidden_conv )
        #self.hidden = nn.Linear(n_hidden, n_classes)

    def forward(self, x):
        if self.embed_mode == 'random':
            x = self.word_embed(x) 
        x = x.unsqueeze(1)
        x = self.conv(x)
        #print(x.size())
        x = F.tanh(x).squeeze(3)
        #print(x.size())
        x = F.max_pool1d(x, x.size(2))
        #print(x.size())
        x = x.squeeze(2)  # max-over-time pooling
        # append external features and feed to fc
        #x = F.tanh(self.combined_feature_vector(x))
        #x = self.dropout(x)
        #x = self.hidden(x)
        #print(x.size())
        return x

class Hierachical_Convolutional_Embedding(nn.Module):
    pass


 

In [12]:
def training_loop(config, model, loss_, optim, training_iter, dev_iter):
    step = 0
    epoch = 0
    total_batches = int(len(train_data) / config['batch_size'])
    while epoch <= config['num_epochs']:
        model.train()
        vectors, labels = get_batch(next(training_iter)) 
        vectors = pad_minibatch(vectors)
        labels = Variable(torch.stack(labels).squeeze())
        model.zero_grad()
        
        
        hidden = model.init_hidden()
        output = model(vectors, hidden)

        lossy = loss_(output, labels)
        lossy.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), 5.0)
        optim.step()
        
        if step % total_batches == 0:
            if epoch % 5 == 0 and epoch!=0:
                print("Epoch %i; Step %i / %i; Train Loss %f; Val Loss: %f; Val acc %f; Val AUC %f" 
                      %(epoch, step, total_batches,lossy.data[0],\
                        evaluate(model, loss_, dev_iter)))
            epoch += 1
        else:
            print("Epoch %i; Step %i / %i; Train Loss %f" %(epoch, step, total_batches ,lossy.data[0]))
            
        step += 1

In [13]:
# Build, initialize, and train model
config = {'vocab_size': len(vocabulary),
          'words_dim': 100,
          'embed_mode': 'random',
          'output_channel': 50,
          'dropout':0.3,
          'target_class':2,
          'note_gru_hidden':20,
          'bidirection_gru': False,
          'batch_size': 8,
          'learning_rate': 0.001,
          'num_epochs':100,
          'filter_width':20
         }

model = AttentionRNN(config)

# Loss and Optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'])

# Train the model
training_iter = data_iter(train_data, config['batch_size'])

dev_iter = eval_iter(val_data, config['batch_size'])

#training_loop(config, model, loss, optimizer, training_iter, dev_iter)

In [14]:
optimizer = torch.optim.Adadelta(model.parameters(), lr=config['learning_rate'])
training_loop(config, model, loss, optimizer, training_iter, dev_iter)

Epoch 1; Step 1 / 939; Train Loss 0.691603
Epoch 1; Step 2 / 939; Train Loss 0.689102
Epoch 1; Step 3 / 939; Train Loss 0.708444
Epoch 1; Step 4 / 939; Train Loss 0.689816
Epoch 1; Step 5 / 939; Train Loss 0.716236
Epoch 1; Step 6 / 939; Train Loss 0.680342
Epoch 1; Step 7 / 939; Train Loss 0.688293
Epoch 1; Step 8 / 939; Train Loss 0.694934
Epoch 1; Step 9 / 939; Train Loss 0.682638
Epoch 1; Step 10 / 939; Train Loss 0.675712
Epoch 1; Step 11 / 939; Train Loss 0.705282
Epoch 1; Step 12 / 939; Train Loss 0.682603
Epoch 1; Step 13 / 939; Train Loss 0.692630
Epoch 1; Step 14 / 939; Train Loss 0.721081
Epoch 1; Step 15 / 939; Train Loss 0.676921
Epoch 1; Step 16 / 939; Train Loss 0.725046
Epoch 1; Step 17 / 939; Train Loss 0.684346
Epoch 1; Step 18 / 939; Train Loss 0.689783
Epoch 1; Step 19 / 939; Train Loss 0.702076
Epoch 1; Step 20 / 939; Train Loss 0.687220
Epoch 1; Step 21 / 939; Train Loss 0.715771
Epoch 1; Step 22 / 939; Train Loss 0.683069
Epoch 1; Step 23 / 939; Train Loss 0.7072

KeyboardInterrupt: 

In [187]:
model.parameters

<bound method Module.parameters of AttentionRNN (
  (note_embedding): Convolutional_Embedding (
    (word_embed): Embedding(181443, 300)
    (static_word_embed): Embedding(181443, 300)
    (nonstatic_word_embed): Embedding(181443, 300)
    (conv): Conv2d(1, 100, kernel_size=(10, 300), stride=(1, 1), padding=(9, 0))
    (dropout): Dropout (p = 0)
  )
  (note_gru): GRU(100, 20)
  (lin_out): Linear (100 -> 2)
)>

In [246]:
def accuracy(out,truth):
    def sm(mat):
        exp = torch.exp(mat)
        sum_exp = exp.sum(1,True)+0.0001
        return exp/sum_exp.expand_as(exp)
    #print(sm(out).size())
    _,max_i = torch.max(sm(out),1)

    eq = torch.eq(max_i,truth).float()
    all_eq = torch.sum(eq)

    return all_eq, all_eq/truth.size(0)*100

In [17]:
truth = Variable(torch.from_numpy(np.array([0,1,0,1,1])))
accuracy(final_out,truth )

NameError: name 'accuracy' is not defined

In [42]:
a = truth.data.numpy()
a == np.array([0,1,0,1,1])

/Users/apple/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  from ipykernel import kernelapp as app


False

In [20]:
list(truth.data.numpy())

[0, 1, 0, 1, 1]

In [34]:
truth = Variable(torch.from_numpy(np.zeros((3,2))))

In [35]:
a = truth.data.numpy()

In [36]:
a[:,0]

array([ 0.,  0.,  0.])

In [37]:
k = list(a)+ list(a)

In [39]:
k = np.array(k)

In [43]:
truth

Variable containing:
 0  0
 0  0
 0  0
[torch.DoubleTensor of size 3x2]

In [45]:
a = torch.nn.CrossEntropyLoss(torch.from_numpy(np.array([[0.3,0.7],[0.5,0.5],[0,1]])), torch.from_numpy(np.array([0,1,1])) )

In [46]:
a

CrossEntropyLoss (
)